# Modeling Dataset 3 using MultiHeadSelfAttention BiLSTM With ADWIN Drift Detection

In [45]:
from keras.layers import (
    Input,
    Dense,
    Dropout,
    LayerNormalization,
    MultiHeadAttention,
    Bidirectional,
    LSTM,
    GlobalAveragePooling1D,
)
from keras.models import Model
from keras.optimizers.legacy import RMSprop
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import os
from keras.callbacks import ModelCheckpoint, Callback
import prettytable
from performance_evaluator.metrics import evaluate
from performance_evaluator.plots import (
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
)
import shutil
from sklearn.model_selection import train_test_split
from river.drift import ADWIN

plt.rcParams["font.family"] = "Space Mono"

In [46]:
def reset_random():
    seed = 1
    import os
    os.environ['PYTHONHASHSEED'] = str(seed)
    import random
    random.seed(seed)
    import numpy as np
    import scipy
    _ = scipy
    np.random.seed(seed)
    import warnings
    warnings.filterwarnings('ignore', category=Warning)
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
    import tensorflow as tf
    tf.compat.v1.random.set_random_seed(seed)
    tf.compat.v1.set_random_seed(seed)


reset_random()

In [47]:
df = pd.read_csv('Data/preprocessed/Dataset3.csv')
df.head()

,processorId,processId,threadId,parentProcessId,hostProcessId,hostThreadId,hostParentProcessId,userId,mountNamespace,pidNamespace,processName,hostName,containerId,eventId,eventName,argsNum,returnValue,Label
0,1.0,0.087239,0.087239,0.085992,0.087239,0.087239,0.085992,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.008933,0
1,1.0,0.086220,0.086220,0.073528,0.086220,0.086220,0.073528,1.0,0.0,0.0,0.015152,0.0,0.0,0.017630,0.03125,0.142857,0.008933,0
2,1.0,0.086220,0.086220,0.073528,0.086220,0.086220,0.073528,1.0,0.0,0.0,0.015152,0.0,0.0,0.248776,0.06250,0.428571,0.008777,0
3,1.0,0.086220,0.086220,0.073528,0.086220,0.086220,0.073528,1.0,0.0,0.0,0.015152,0.0,0.0,0.017630,0.03125,0.142857,0.008933,0
4,1.0,0.086220,0.086220,0.073528,0.086220,0.086220,0.073528,1.0,0.0,0.0,0.015152,0.0,0.0,0.983350,0.09375,0.000000,0.008933,0


In [48]:
from performance_evaluator.make_cls import make_cls
CLASSES = ['Normal', 'Attack']
x, y = df.values[:, :-1], df.values[:, -1]
# x, y = make_cls(df, 'Label', CLASSES)
x = np.expand_dims(x, axis=1)
y_cat = to_categorical(y, len(CLASSES))

In [49]:
def buildModel(time_steps, features, num_classes):
    inputs = Input(shape=(time_steps, features))

    # ---- BiLSTM Encoder ----
    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = LayerNormalization()(x)

    # ---- Multi-Head Self Attention ----
    attn = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)

    x = x + attn
    x = LayerNormalization()(x)

    # ---- Temporal Aggregation ----
    x = GlobalAveragePooling1D()(x)

    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)

    model.compile(
        optimizer=RMSprop(0.000002),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [50]:
RESULTS_PLOT = {
    "Train": {
        "CONF_MAT": plt.figure(num=1, figsize=(3, 3)),
        "PR_CURVE": plt.figure(num=2),
        "ROC_CURVE": plt.figure(num=3),
    },
    "Test": {
        "CONF_MAT": plt.figure(num=4, figsize=(3, 3)),
        "PR_CURVE": plt.figure(num=5),
        "ROC_CURVE": plt.figure(num=6),
    },
}
ACC_PLOT = plt.figure(num=7)
LOSS_PLOT = plt.figure(num=8)

<Figure size 300x300 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 300x300 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [56]:
def print_df_to_table(df, p=True):
    field_names = list(df.columns)
    p_table = prettytable.PrettyTable(field_names=field_names)
    p_table.add_rows(df.values.tolist())
    d = "\n".join(
        ["\t\t{0}".format(p_) for p_ in p_table.get_string().splitlines(keepends=False)]
    )
    if p:
        print(d)
    return d


def plot_line(plt_, y1, y2, epochs, for_, save_path):
    ax = plt_.gca()
    ax.clear()
    ax.plot(range(epochs), y1, label="Training", color="dodgerblue")
    ax.plot(range(epochs), y2, label="Validation", color="orange")
    ax.set_title("Training and Validation {0}".format(for_))
    ax.set_xlabel("Epochs")
    ax.set_ylabel(for_)
    ax.set_xlim([0, epochs])
    ax.legend()
    plt_.tight_layout()
    plt_.savefig(save_path)


def plot_acc_loss(df, plt1, plt2, save_dir):
    epochs = len(df)
    acc = df["accuracy"].values
    val_acc = df["val_accuracy"].values
    loss = df["loss"].values
    val_loss = df["val_loss"].values
    plot_line(
        plt1, acc, val_acc, epochs, "Accuracy", os.path.join(save_dir, "accuracy.png")
    )
    plot_line(plt2, loss, val_loss, epochs, "Loss", os.path.join(save_dir, "loss.png"))


def plot(y, pred, prob, plts, results_dir):
    for_ = os.path.basename(results_dir)
    print("\t[INFO] Evaluating {0} Data".format(for_))
    os.makedirs(results_dir, exist_ok=True)
    from sklearn.metrics import (
        accuracy_score,
        precision_score,
        recall_score,
        f1_score,
        matthews_corrcoef,
    )

    df = pd.DataFrame(
        [
            ["Acucuracy", accuracy_score(y, pred)],
            ["Precision", precision_score(y, pred)],
            ["Recall", recall_score(y, pred)],
            ["F1-Score", f1_score(y, pred)],
            ["MCC", matthews_corrcoef(y, pred)],
        ],
        columns=["Metrics", "Values"],
    )
    df["Values"] = df["Values"].round(decimals=4)
    df.to_csv(os.path.join(results_dir, "metrics.csv"), index=False)
    print_df_to_table(df)

    fig = plts[for_]["CONF_MAT"]
    ax = fig.gca()
    ax.clear()
    confusion_matrix(
        y,
        pred,
        CLASSES,
        ax=ax,
        title="{0}ing Phase - Confusion matrix".format(for_),
    )
    fig.tight_layout()
    fig.savefig(os.path.join(results_dir, "conf_mat.png"))
    fig.show()

    fig = plts[for_]["PR_CURVE"]
    ax = fig.gca()
    ax.clear()
    precision_recall_curve(
        y,
        prob,
        CLASSES,
        ax=ax,
        legend_ncol=2,
        title="Precision-Recall Curve",
    )
    fig.tight_layout()
    fig.savefig(os.path.join(results_dir, "pr_curve.png"))
    fig.show()

    fig = plts[for_]["ROC_CURVE"]
    ax = fig.gca()
    ax.clear()
    roc_curve(
        y,
        prob,
        CLASSES,
        ax=ax,
        legend_ncol=2,
        title="ROC Curve",
    )
    fig.tight_layout()
    fig.savefig(os.path.join(results_dir, "roc_curve.png"))
    fig.show()


class TrainingCallback(Callback):
    def __init__(self, acc_loss_path, plt1, plt2):
        self.acc_loss_path = acc_loss_path
        self.plt1 = plt1
        self.plt2 = plt2
        if os.path.isfile(self.acc_loss_path):
            self.df = pd.read_csv(self.acc_loss_path)
            plot_acc_loss(
                self.df, self.plt1, self.plt2, os.path.dirname(self.acc_loss_path)
            )
        else:
            self.df = pd.DataFrame(
                [], columns=["epoch", "accuracy", "val_accuracy", "loss", "val_loss"]
            )
            self.df.to_csv(self.acc_loss_path, index=False)
        Callback.__init__(self)

    def on_epoch_end(self, epoch, logs=None):
        self.df.loc[len(self.df.index)] = [
            int(epoch + 1),
            round(logs["accuracy"], 4),
            round(logs["val_accuracy"], 4),
            round(logs["loss"], 4),
            round(logs["val_loss"], 4),
        ]
        self.df.to_csv(self.acc_loss_path, index=False)
        plot_acc_loss(
            self.df, self.plt1, self.plt2, os.path.dirname(self.acc_loss_path)
        )


class ADWINDriftCallback(Callback):

    def __init__(self, monitor="val_loss"):
        super().__init__()
        self.monitor = monitor
        self.adwin = ADWIN()

    def on_epoch_end(self, epoch, logs=None):

        value = logs.get(self.monitor)

        if value is None:
            return

        in_drift, _ = self.adwin.update(value)

        if in_drift:
            print("\n⚠️ Concept Drift Detected by ADWIN")

            # Example reaction: reduce learning rate
            lr = self.model.optimizer.learning_rate.numpy()
            new_lr = lr * 0.5
            self.model.optimizer.learning_rate.assign(new_lr)

            print(f"Learning rate reduced → {new_lr}")

In [52]:
model_dir = "models/Dataset3"
if os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
os.makedirs(model_dir, exist_ok=True)
acc_loss_csv_path = os.path.join(model_dir, "acc_loss.csv")
model_path = os.path.join(model_dir, "model.h5")
training_cb = TrainingCallback(acc_loss_csv_path, ACC_PLOT, LOSS_PLOT)
checkpoint = ModelCheckpoint(
    model_path,
    save_best_only=True,
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=False,
)
model = buildModel(x.shape[1], x.shape[2], len(CLASSES))
initial_epoch = 0
if os.path.isfile(model_path) and os.path.isfile(acc_loss_csv_path):
    print("[INFO] Loading Pre-Trained Model :: {0}".format(model_path))
    model.load_weights(model_path)
    initial_epoch = len(pd.read_csv(acc_loss_csv_path))
print("[INFO] Fitting Data")
model.fit(
    x,
    y_cat,
    validation_data=(x, y_cat),
    batch_size=4096,
    epochs=50,
    verbose=1,
    initial_epoch=initial_epoch,
    callbacks=[training_cb, checkpoint],
)
model.load_weights(model_path)

[INFO] Fitting Data
Epoch 1/50
28/28 [==============================] - 6s 113ms/step - loss: 1.3440 - accuracy: 0.5035 - val_loss: 1.2378 - val_accuracy: 0.4889
Epoch 2/50
28/28 [==============================] - 2s 90ms/step - loss: 1.2911 - accuracy: 0.5070 - val_loss: 1.1890 - val_accuracy: 0.4889
Epoch 3/50
28/28 [==============================] - 2s 86ms/step - loss: 1.2518 - accuracy: 0.5102 - val_loss: 1.1411 - val_accuracy: 0.5032
Epoch 4/50
28/28 [==============================] - 2s 85ms/step - loss: 1.2094 - accuracy: 0.5129 - val_loss: 1.0953 - val_accuracy: 0.5095
Epoch 5/50
28/28 [==============================] - 2s 85ms/step - loss: 1.1667 - accuracy: 0.5171 - val_loss: 1.0512 - val_accuracy: 0.5166
Epoch 6/50
28/28 [==============================] - 2s 87ms/step - loss: 1.1262 - accuracy: 0.5223 - val_loss: 1.0088 - val_accuracy: 0.5170
Epoch 7/50
28/28 [==============================] - 2s 85ms/step - loss: 1.0876 - accuracy: 0.5277 - val_loss: 0.9684 - val_accuracy:

In [57]:
print("[INFO] Evaluating Training|Testing ==> 70:30")
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, shuffle=True, random_state=1
)
train_prob = model.predict(train_x, verbose=False)
train_pred = np.argmax(train_prob, axis=1).ravel().astype(int)
plot(
    train_y.ravel().astype(int),
    train_pred,
    train_prob,
    RESULTS_PLOT,
    "results/Dataset3/Train",
)
test_prob = model.predict(test_x, verbose=False)
test_pred = np.argmax(test_prob, axis=1).ravel().astype(int)
plot(
    test_y.ravel().astype(int),
    test_pred,
    test_prob,
    RESULTS_PLOT,
    "results/Dataset3/Test",
)

[INFO] Evaluating Training|Testing ==> 70:30
	[INFO] Evaluating Train Data
		+-----------+--------+
		|  Metrics  | Values |
		+-----------+--------+
		| Acucuracy | 0.9869 |
		| Precision | 0.9999 |
		|   Recall  | 0.9748 |
		|  F1-Score | 0.9872 |
		|    MCC    | 0.9741 |
		+-----------+--------+
	[INFO] Evaluating Test Data
		+-----------+--------+
		|  Metrics  | Values |
		+-----------+--------+
		| Acucuracy | 0.9869 |
		| Precision |  1.0   |
		|   Recall  | 0.9749 |
		|  F1-Score | 0.9873 |
		|    MCC    | 0.9742 |
		+-----------+--------+
